In [1]:
#!pip install pyarrow
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import neptune
from datetime import datetime
from pytz import timezone

/opt/homebrew/lib/python3.11/site-packages/neptune/internal/backends/hosted_client.py:50: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version


In [2]:
current_directory = os.getcwd()
file_path = os.path.join(current_directory, 'data', 'train_series.parquet')
train_series = pd.read_parquet(file_path, engine='pyarrow')

file_path = os.path.join(current_directory, 'data', 'test_series.parquet')
test_series = pd.read_parquet(file_path, engine='pyarrow')
file_path = os.path.join(current_directory, 'data', 'sample_submission.csv')

sample_submission = pd.read_csv(file_path)
file_path = os.path.join(current_directory, 'data', 'train_events.csv')
train_events = pd.read_csv(file_path)


In [3]:
print(train_series.head())
print(train_series.info())
print(train_series.describe())


      series_id  step                 timestamp  anglez    enmo
0  038441c925bb     0  2018-08-14T15:30:00-0400  2.6367  0.0217
1  038441c925bb     1  2018-08-14T15:30:05-0400  2.6368  0.0215
2  038441c925bb     2  2018-08-14T15:30:10-0400  2.6370  0.0216
3  038441c925bb     3  2018-08-14T15:30:15-0400  2.6368  0.0213
4  038441c925bb     4  2018-08-14T15:30:20-0400  2.6368  0.0215
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127946340 entries, 0 to 127946339
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   series_id  object 
 1   step       uint32 
 2   timestamp  object 
 3   anglez     float32
 4   enmo       float32
dtypes: float32(2), object(2), uint32(1)
memory usage: 3.3+ GB
None
               step        anglez          enmo
count  1.279463e+08  1.279463e+08  1.279463e+08
mean   2.548048e+05 -8.810453e+00  4.131503e-02
std    1.778930e+05  3.552188e+01  1.018289e-01
min    0.000000e+00 -9.000000e+01  0.000000e+00
25%    1.158120e+05 -3.1

In [4]:
print(train_events.head())
print(train_events.info())
print(train_events.describe())

      series_id  night   event     step                 timestamp
0  038441c925bb      1   onset   4992.0  2018-08-14T22:26:00-0400
1  038441c925bb      1  wakeup  10932.0  2018-08-15T06:41:00-0400
2  038441c925bb      2   onset  20244.0  2018-08-15T19:37:00-0400
3  038441c925bb      2  wakeup  27492.0  2018-08-16T05:41:00-0400
4  038441c925bb      3   onset  39996.0  2018-08-16T23:03:00-0400
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14508 entries, 0 to 14507
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   series_id  14508 non-null  object 
 1   night      14508 non-null  int64  
 2   event      14508 non-null  object 
 3   step       9585 non-null   float64
 4   timestamp  9585 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 566.8+ KB
None
              night           step
count  14508.000000    9585.000000
mean      15.120072  214352.123944
std       10.286758  141268.408192
min    

In [5]:
import neptune

run = neptune.init_run(
    project="w25536/Detect-Sleep-States",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI2ZGUxZWY5Yi01YTdjLTQ5YjEtYjQ5YS02OGYyMjA1NDMyMzkifQ==",
)  # your credentials

params = {"learning_rate": 0.001, "optimizer": "Adam"}
run["parameters"] = params

for epoch in range(10):
    run["train/loss"].append(0.9 ** epoch)

run["eval/f1_score"] = 0.66

run.stop()

/var/folders/hb/_y1g8w4d6j910_w7_rr3vx700000gn/T/ipykernel_88543/4001559911.py:3: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  run = neptune.init_run(


https://app.neptune.ai/w25536/Detect-Sleep-States/e/DET-2
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 13 operations to synchronize with Neptune. Do not kill this process.
All 13 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/w25536/Detect-Sleep-States/e/DET-2/metadata


In [6]:
train_series.isna().sum()

series_id    0
step         0
timestamp    0
anglez       0
enmo         0
dtype: int64

In [7]:
train_series['timestamp'] = pd.to_datetime(train_series['timestamp']).apply(lambda t : t.tz_localize(None))

/var/folders/hb/_y1g8w4d6j910_w7_rr3vx700000gn/T/ipykernel_88543/2568038629.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  train_series['timestamp'] = pd.to_datetime(train_series['timestamp']).apply(lambda t : t.tz_localize(None))
